<a href="https://colab.research.google.com/github/akashpatil1996/Face_mask_detector/blob/main/Face_Mask_Detector%F0%9F%98%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the dataset from kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d omkargurav/face-mask-dataset

100% 163M/163M [00:08<00:00, 22.5MB/s]
100% 163M/163M [00:08<00:00, 21.2MB/s]


In [3]:
# Importing required Libraries
import os
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
# Unzipping the data
import zipfile
zip_ref = zipfile.ZipFile('/content/face-mask-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
# Create the labels and checking the number of images
without_mask_files = os.listdir('/content/data/without_mask')
with_mask_files = os.listdir('/content/data/with_mask')

print(len(with_mask_files), 'images with mask')
print(len(without_mask_files), 'images without mask')

with_mask_labels = [1]*3725
without_mask_labels = [0]*3828

3725 images with mask
3828 images without mask


In [6]:
# Convert images to numpy arrays and appending them to a list
with_mask_path = '/content/data/with_mask/'
without_mask_path = '/content/data/without_mask/'

data = []

for img_file in with_mask_files:
  image = Image.open(with_mask_path + img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)


for img_file in without_mask_files:
  image = Image.open(without_mask_path + img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [7]:
# Creating the labels
with_mask_labels = [1]*3725
without_mask_labels = [0]*3828
labels = with_mask_labels + without_mask_labels

In [8]:
# Converting list to numpy array
X = np.array(data)
y = np.array(labels)
print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')

Shape of X: (7553, 128, 128, 3)
Shape of y: (7553,)


In [9]:
# Normalizing the data
X = X/255

In [10]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2)

print(f'Train Data: {X_train.shape}')
print(f'Test Data: {X_test.shape}')

Train Data: (6042, 128, 128, 3)
Test Data: (1511, 128, 128, 3)


In [11]:
# Model Building
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='sigmoid'))

In [12]:
# model.summary()

In [13]:
# Compiling the model
model.compile(optimizer='Adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

In [15]:
result = model.fit(
    X_train,
    y_train,
    epochs = 20,
    validation_data = (X_test,y_test),
    callbacks=[early_stopping]

)

Epoch 1/20
189/189 [==============================] - 16s 27ms/step - loss: 0.4930 - accuracy: 0.7749 - val_loss: 0.3717 - val_accuracy: 0.8637
Epoch 2/20
189/189 [==============================] - 4s 20ms/step - loss: 0.3067 - accuracy: 0.8757 - val_loss: 0.2465 - val_accuracy: 0.8974
Epoch 3/20
189/189 [==============================] - 4s 21ms/step - loss: 0.2502 - accuracy: 0.9042 - val_loss: 0.1992 - val_accuracy: 0.9173
Epoch 4/20
189/189 [==============================] - 3s 18ms/step - loss: 0.2190 - accuracy: 0.9167 - val_loss: 0.1952 - val_accuracy: 0.9219
Epoch 5/20
189/189 [==============================] - 3s 18ms/step - loss: 0.1929 - accuracy: 0.9230 - val_loss: 0.1902 - val_accuracy: 0.9226
Epoch 6/20
189/189 [==============================] - 4s 21ms/step - loss: 0.1617 - accuracy: 0.9356 - val_loss: 0.1627 - val_accuracy: 0.9358
Epoch 7/20
189/189 [==============================] - 3s 18ms/step - loss: 0.1443 - accuracy: 0.9439 - val_loss: 0.1831 - val_accuracy: 0.931

In [16]:
# Saving the model
model.save('FaceMaskDetModel.h5')